# Remote Flood Mapping

Besides processing of a flood's extent on your own machine one can also offload the processing graph to the EODC Dask Gateway. 

This notebook is based on the tutorial to connect to EODC Dask available at [this link](https://github.com/eodcgmbh/eodc-examples/blob/main/demos/dask.ipynb).

More information about EODC Dask Gateway can be found [here](https://docs.eodc.eu/services/dask.html).


## Connect to EODC Dask

Autentication is required through an username and password that should be requested to EODC.

In [ ]:
from eodc.dask import EODCDaskGateway
from rich.console import Console
from rich.prompt import Prompt
console = Console()
your_username = Prompt.ask(prompt="Enter your Username")
gateway = EODCDaskGateway(username=your_username)

### Cluster Configuration

It is possible to change the default configuration to optimize your specifications at the cell below.

In [ ]:
cluster_options = gateway.cluster_options()
cluster_options

### Create Dask Cluster

#### Initiate a client
In order to create a Dask cluster, it is first necessary to initiate the client. 

<div class="alert alert-block alert-info">
<b>Important:</b> Per default no worker is spawned, therefore please use the widget to add/scale Dask workers in other to enable compuations on the cluster.
</div>

In [ ]:
cluster = gateway.new_cluster(cluster_options)
client = cluster.get_client()
cluster

Another way to spawn workers is trhough the cell below that sets two workers as starting point and enables up to 5 workers, if necessary.

In [ ]:
cluster.adapt(minimum=2, maximum=5)

#### List available clusters

In [ ]:
console.print(gateway.list_clusters())

#### Connect to already running clusters 

Instead of restarting the whole process again, you can directly connect to cluster already running.

In [ ]:
cluster = gateway.connect(gateway.list_clusters()[0].name)
console.print(cluster)

### Dask dashboard

The following link displays the Dask Dashboard that can be used to monitor the execution of computations.

In [ ]:
cluster.dashboard_link

## Remote Dask Flood Mapping

Uisng the same example from the local dask setup, the event chosen is storm Babet which hit the Danish and Northern coast of Germany at the 20<sup>th</sup> of October 2023 [Wikipedia](https://en.wikipedia.org/wiki/Storm_Babet). We target an area around Zingst at the Baltic coast of Northern Germany.

In [ ]:
from dask_flood_mapper import flood

In [ ]:
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
fd = flood.probability(bbox=bbox, datetime=time_range).compute()
fd

In [ ]:
import hvplot.xarray

fd.hvplot.quadmesh(
    x="x",
    y="y",
    rasterize=True,
    geo=True,
).opts(frame_height=400)

Make sure to shutdown the cluster.

In [ ]:
cluster.close(shutdown=True)